In [25]:
%pip install --upgrade selenium

Note: you may need to restart the kernel to use updated packages.


In [46]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.common.by import By
from datetime import date,timedelta
from dateutil.parser import parse

In [27]:
# Daily Discission Threads
url = "https://www.reddit.com/r/wallstreetbets/search?q=flair_name%3A%22Daily%20Discussion%22&restrict_sr=1&sort=new"
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications" : 2}
chrome_options.add_experimental_option("prefs",prefs)

In [118]:
#Functions
def select_sort_filter(filtr:str = "New"):
    """
    Select how you want the results to be filtered.
    Options are New, Hot, Top, Relevance, and Most Comments
    """
    filter_select_button = driver.find_element_by_xpath('//button[@data-testid = "search-results-filter-sort"]')
    filter_select_button.click()
    
    filter_button = driver.find_element_by_xpath(f'//button[@role = "menuitem"]//span[text()="{filtr}"]')
    filter_button.click()
def pull_Comments(filtr_comments: str = "top"):
    comments = []
    
    comments_sort = driver.find_element(By.XPATH,('//span[@class="_2-cXnP74241WI7fpcpfPmg _3LRBCA71BwLLXBNsSlY7HW"]'))
    comments_sort.click()
    time.sleep(2)
    
    comments_sort_select = driver.find_element(By.XPATH,(f'//button[@class="_10K5i7NW6qcm-UoCtpB3aK _3LwUIE7yX7CZQKmD2L87vf _1oYEKCssGFjqxQ9jJMNj5G"]//span[text() ="{filtr_comments}"]'))
    comments_sort_select.click()
    time.sleep(5)
    
    sorted_ = driver.find_element(By.XPATH,('//span[@class="_2-cXnP74241WI7fpcpfPmg _3LRBCA71BwLLXBNsSlY7HW"]'))
    sorted_by = "".join(sorted_.text.split(' ')[-1:])
    
    if sorted_by.lower() != filtr_comments:
            print('comments not properly sorted')
    
    soup = BeautifulSoup(driver.page_source)
    comment_cells = soup.find('div', class_ = "_1YCqQVO-9r-Up6QPB9H6_4 _1YCqQVO-9r-Up6QPB9H6_4").find_all('div', class_ = "_3sf33-9rVAO_v4y0pIW_CH")
        
    for ix, cell in enumerate(comment_cells):
        comment = {}
        comment_id = cell.get('id')
        
        if 'moreComments' not in comment_id:
            comment.update({'comment_id':comment_id})

            username = cell.find('div', class_= "_2mHuuvyV9doV3zwbZPtIPG").find('a')
            comment.update({'username':username.text})

            comment_body =cell.find('p')
            comment.update({'body':comment_body.text})

            comment_upvotes = cell.find('div', class_ = "_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ _3ChHiOyYyUkpZ_Nm3ZyM2M")
            comment.update({"upvotes":comment_upvotes.text})
        
            comments.append(comment)
        else:
            ix -=1
        if ix == 2:
            break
    return comments    
    
def pull_Posts(filtr_comments: str = "top"):
    """
    filtr_comments takes best, top, new, controversial, old, or q&a
    """
    
    posts = []
    
    links = driver.find_elements(By.XPATH,('//h3'))
    
    for ix,link in enumerate(links):
        post = {}
        post.update({'title':link.text}) 
        date = "".join(link.text.split(' ')[-3:])
        post.update({"Reference_Date":parse(date)})
        
        link.click()
        time.sleep(5)
        
        post_body = driver.find_element(By.XPATH,('//p[@class="_1qeIAgB0cPwnLhDF9XSiJM"]'))
        post.update({'body':post_body.text})
        time.sleep(5)
        
        num_comments = driver.find_element(By.XPATH,('//span[@class="FHCV02u6Cp2zYL0fhQPsO"]'))
        post.update({'num_comments': num_comments.text})
        time.sleep(2)
        nums_upvotes_rev = driver.find_elements(By.XPATH,('//span[@class="WBY6J5DJsZFZXSxBqtq0F"]//span'))
        if nums_upvotes_rev:
            nums_upvotes = [num.text for num in reversed(nums_upvotes_rev)]
            
            num_upvotes = ''.join(nums_upvotes)
            post.update({'num_upvotes':num_upvotes})
        else:
            num_upvotes = driver.find_element(By.XPATH,('//div[@class="_1rZYMD_4xY3gRcSS3p8ODO _25IkBM0rRUqWX5ZojEMAFQ"]'))
            post.update({'num_upvotes':num_upvotes.text})
        #time.sleep(5)
        
        post_author = driver.find_element(By.XPATH,('//a[@class="_2tbHP6ZydRpjI44J3syuqC  _23wugcdiaj44hdfugIAlnX oQctV4n0yUb0uiHDdGnmE"]'))
        post.update({'author':post_author.text})
        
        comments = pull_Comments(filtr_comments)
        
        post.update({'comments':comments})
        
        close_button = driver.find_element(By.XPATH,('//button[@title = "Close"]'))
        close_button.click()
        close_button.click()
        
        posts.append(post)
        
        time.sleep(5)
        if ix == 2:
            break
    return(posts)
        

In [120]:
global driver
driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get(url)
select_sort_filter("Hot")
time.sleep(10)
posts_list = pull_Posts()
driver.quit()

<ipython-input-120-e38e51968628>:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options=chrome_options)
<ipython-input-118-ce5d2c634eb7>:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  filter_select_button = driver.find_element_by_xpath('//button[@data-testid = "search-results-filter-sort"]')
<ipython-input-118-ce5d2c634eb7>:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  filter_button = driver.find_element_by_xpath(f'//button[@role = "menuitem"]//span[text()="{filtr}"]')


In [121]:
posts_list

[{'title': 'What Are Your Moves Tomorrow, February 09, 2022',
  'Reference_Date': datetime.datetime(2022, 2, 9, 0, 0),
  'body': 'Your daily trading discussion thread. Please keep the shitposting to a minimum.',
  'num_comments': '9.7k Comments',
  'num_upvotes': '271',
  'author': 'u/AutoModerator',
  'comments': [{'comment_id': 't1_hw5c7qx',
    'username': '',
    'body': "Checking a stock after you sold feels like looking at your ex's social media",
    'upvotes': '131'},
   {'comment_id': 't1_hw4vnxg',
    'username': '',
    'body': 'Chipotle is Mexican food for people who hate Mexicans',
    'upvotes': '124'}]},
 {'title': 'Daily Discussion Thread for February 08, 2022',
  'Reference_Date': datetime.datetime(2022, 2, 8, 0, 0),
  'body': 'Your daily trading discussion thread. Please keep the shitposting to a minimum.',
  'num_comments': '15.5k Comments',
  'num_upvotes': '330',
  'author': 'u/AutoModerator',
  'comments': [{'comment_id': 't1_hw2refo',
    'username': '',
    'bod